<a href="https://colab.research.google.com/github/agarwalsourabh55/huggingface-tapas/blob/main/huggingface_tapas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch-scatter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Different Types of Declaring TAPAS models


1.   Default SQA Configutaion
2.   with WTQ Configuration
3.   With WikiSQL Configuration


In [6]:
from transformers import TapasConfig, TapasForQuestionAnswering

# for example, the base sized model with default SQA configuration
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base")

# or, the base sized model with WTQ configuration
config = TapasConfig.from_pretrained("google/tapas-base-finetuned-wtq")
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base", config=config)

# # or, the base sized model with WikiSQL configuration
# config = TapasConfig( "google/tapas-base-finetuned-wikisql-supervised",
#                         num_aggregation_labels=3,
#                         average_logits_per_cell=True)
# model = TapasForQuestionAnswering.from_pretrained("google/tapas-base", config=config)

Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['column_output_weights', 'column_output_bias', 'output_bias', 'output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['column_output_weights', 'aggregation_classifier.weight', 'output_bias', 'aggregation_classifier.bias', 'column_output_bias', 'output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



### Defining the Tokenizer class


1.   Defining the data labels.
2.   Defining the querie.
3.   Defining the answers of that queries.
4.   Converting all into the DAtaframe if not.
5.   Tokenizer Takes Text Only. 
6.   Advised to use Dataloaders for training the TAPAS



In [7]:
from transformers import TapasTokenizer
import pandas as pd

model_name = "google/tapas-base"
tokenizer = TapasTokenizer.from_pretrained(model_name)

data = {"Actors": ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"], "Number of movies": ["87", "53", "69"]}
queries = [
    "What is the name of the first actor?",
    "How many movies has George Clooney played in?",
    "What is the total number of movies?",
]
answer_coordinates = [[(0, 0)], [(2, 1)], [(0, 1), (1, 1), (2, 1)]]
answer_text = [["Brad Pitt"], ["69"], ["209"]]
table = pd.DataFrame.from_dict(data)
inputs = tokenizer(
    table=table,
    queries=queries,
    answer_coordinates=answer_coordinates,
    answer_text=answer_text,
    padding="max_length",
    return_tensors="pt",
)
inputs

{'input_ids': tensor([[ 101, 2054, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2116,  ...,    0,    0,    0],
        [ 101, 2054, 2003,  ...,    0,    0,    0]]), 'labels': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'numeric_values': tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]]), 'numeric_values_scale': tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]]), 'token_type_ids': tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

In [9]:
table

,Actors,Number of movies
0,Brad Pitt,87
1,Leonardo Di Caprio,53
2,George Clooney,69


## TAPAS for Question Answering pretrained

Using Different methodology to defint the predefine the logits

In [10]:
from transformers import TapasTokenizer, TapasForQuestionAnswering
import pandas as pd

tokenizer = TapasTokenizer.from_pretrained("google/tapas-base-finetuned-wtq")
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base-finetuned-wtq")

data = {
    "Actors": ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"],
    "Age": ["56", "45", "59"],
    "Number of movies": ["87", "53", "69"],
}
table = pd.DataFrame.from_dict(data)
queries = ["How many movies has George Clooney played in?", "How old is Brad Pitt?"]

inputs = tokenizer(table=table, queries=queries, padding="max_length", return_tensors="pt")
outputs = model(**inputs)

logits = outputs.logits
logits_aggregation = outputs.logits_aggregation

### Taking the Inference of TAPAS Pretrained models using the cricketers data.
Conducted a small POC to check how accurately TAPAS model used the predefined modes and predict the modelling approach

In [18]:
import pandas as pd
data = pd.read_csv('/content/cricket_table.csv')
data = data.astype(str)

In [20]:
data

,Pos,Player,Team,Span,Innings,Runs,Highest Score,Average,Strike Rate
0,1,Sachin Tendulkar,India,1989-2012,452,18426,200,44.83,86.23
1,2,Kumar Sangakkara,Sri Lanka,2000-2015,380,14234,169,41.98,78.86
2,3,Ricky Ponting,Australia,1995-2012,365,13704,164,42.03,80.39
3,4,Sanath Jayasuriya,Sri Lanka,1989-2011,433,13430,189,32.36,91.2
4,5,Mahela Jayawardene,Sri Lanka,1998-2015,418,12650,144,33.37,78.96
5,6,Virat Kohli,India,2008-2020,236,11867,183,59.85,93.39
6,7,Inzamam-ul-Haq,Pakistan,1991-2007,350,11739,137,39.52,74.24
7,8,Jacques Kallis,South Africa,1996-2014,314,11579,139,44.36,72.89
8,9,Saurav Ganguly,India,1992-2007,300,11363,183,41.02,73.7
9,10,Rahul Dravid,India,1996-2011,318,10889,153,39.16,71.24


In [19]:
# Defined the tapas model and the pipelinein which we are going to use it.
from transformers import pipeline
tqa = pipeline(task="table-question-answering", 
               model="google/tapas-base-finetuned-wtq")

In [25]:
qureies = ['what is the span of Saurav Ganguly',
           "Who has the Highest score",
           "what is the sum of total runs from the data",
           "Which Player playes for team India"]
answer_list = tqa(table=data, query=qureies)

In [26]:
for answer in answer_list:
    print(answer["answer"])

1992-2007
Sachin Tendulkar
SUM > 18426, 14234, 13704, 13430, 12650, 11867, 11739, 11579, 11363, 10889
Sachin Tendulkar, Virat Kohli, Saurav Ganguly, Rahul Dravid


In [27]:
answer

{'answer': 'Sachin Tendulkar, Virat Kohli, Saurav Ganguly, Rahul Dravid',
 'coordinates': [(0, 1), (5, 1), (8, 1), (9, 1)],
 'cells': ['Sachin Tendulkar',
  'Virat Kohli',
  'Saurav Ganguly',
  'Rahul Dravid'],
 'aggregator': 'NONE'}